In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive/MyDrive/CSE455\ Computer\ Vision\ Win23

/content/drive/MyDrive/CSE455 Computer Vision Win23


In [3]:
ls

 Combined_Frames/
'Copy of 01 - Computer Vision: Introduction.gslides'
 SampleVideo1_Frames/
 SampleVideo1_Frames_Interpolated/
 SampleVideo1.mp4
 test1/
 test2/
 VideoFrameInterpolationWithOpticalFlow.ipynb


# CSE 455 Computer Vision Final Project: Video Frame Rate Interpolation with Farneback Optical Flow
## Author: Ciel Sun
## Date: March 10, 2023
## Overview: 
## This script will perform a video frame interpolation using the technique of optical flow. The generated video sequence will have a frame rate about twice that of the original/source video sequence(i.e. original frame rate 30fps---> new frame rate 59fps).



In [4]:
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow

## 1. Decompose a video sequence into individual frame images

In [17]:
ls

 Combined_Frames/
'Copy of 01 - Computer Vision: Introduction.gslides'
 SampleVideo1_Frames/
 SampleVideo1_Frames_Interpolated/
 SampleVideo1.mp4
 test1/
 test2/
 VideoFrameInterpolationWithOpticalFlow.ipynb


In [40]:
im_path = "SampleVideo1_Frames"
if not os.path.exists(im_path):
  os.mkdir(im_path)
video_name = "SampleVideo1.mp4"
capture = cv2.VideoCapture(video_name)

success, frame1 = capture.read()
count = 1                                                                       # starting with "frame1.jpg"
if success: cv2.imwrite(im_path + f"/frame{count}-0.jpg", frame1)               # i.e. 'frame1-0.jpg', 'frame2-0.jpg'
fps_og = capture.get(cv2.CAP_PROP_FPS)                                          # original frame rate per second 25
# while success:
#   success, im = capture.read()
#   if success:
#     count += 1
#     cv2.imwrite(im_path + f"/frame{count}-0.jpg", im)                           # frames are saved as JPEG files
#     if cv2.waitKey(10) == 27:                                                 # exit if Escape is hit
#       break
totalFrames = len(os.listdir(im_path))
print(f'{totalFrames} frames decomposed from video {video_name}')
print(f'Original frame rate is {fps_og} fps')



175 frames decomposed from video SampleVideo1.mp4
Original frame rate is 25.0 fps


## 2. Generate interpolated frames

In [10]:
def interpolate(prvs, next):
  '''
  Generate and return a mid frame based on Farneback opitcal flow of the previous frame and next frame
  Parameters:
    prvs: previous frame as ndarray with BGR format
    next: next frame as ndarray with BGR format
  Return:
    mid_frame: generated middle frame between previous and next frame
  '''
  y_max = prvs.shape[0] - 1
  x_max = prvs.shape[1] - 1
  prvs_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
  next_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
  # Use gray images to calculate flow with dimension: n x m x 2; 2 channels:[dx/dt, dy/dt]
  flow = cv2.calcOpticalFlowFarneback(prvs_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
  mid_frame = prvs.copy()
  for y, row in enumerate(prvs):
    for x, col in enumerate(row):
      for z, pixVal in enumerate(col):
        new_y = y + int(flow[y, x, 1])
        new_x = x + int(flow[y, x, 0])
        new_y = new_y if new_y < y_max else y_max
        new_x = new_x if new_x < x_max else x_max
        mid_frame[y, x, z] = prvs[new_y, new_x, z]
  return mid_frame

In [26]:
from natsort import natsorted, ns
file_list = natsorted(os.listdir(im_path))

out_path = im_path + "_Interpolated"                                            # a path storing all interpolated frames
if not os.path.exists(out_path):
  os.mkdir(out_path)

# for i in range(len(file_list) - 1):                                             # stop at the second last frame
# # for i in range(3):
#   prvs = cv2.imread(im_path + "/" + file_list[i])
#   next = cv2.imread(im_path + "/" + file_list[i+1])
#   mid_frame = interpolate(prvs, next)
#   cv2.imwrite(out_path + f"/frame{i+1}-1.jpg", mid_frame)                       # i.e. 'frame1-1.jpg','frame2-1.jpg'...


## 3. Compose the original frames and interpolated frames back into one video sequence

In [55]:
mkdir Combined_Frames

mkdir: cannot create directory ‘Combined_Frames’: File exists


In [19]:
cp SampleVideo1_Frames_Interpolated/*.jpg Combined_Frames

In [20]:
cp SampleVideo1_Frames/*.jpg Combined_Frames

In [43]:
# Origianl frames and interpolation frames are both copied to directory "Combined_Frames"
interpolate_list = os.listdir(out_path)
combine_path = "Combined_Frames"
combine_list = natsorted(os.listdir("Combined_Frames"))
print(f"Original number of frames:{len(file_list)}")
print(f"Number of frames generated: {len(interpolate_list)}")
print(f"Total number of frames: {len(combine_list)}")

Original number of frames:175
Number of frames generated: 174
Total number of frames: 349


In [41]:
im_arr = []
h, w, c = frame1.shape
size = (w, h)
for fname in combine_list:
  im_path = os.path.join(combine_path, fname)
  im = cv2.imread(im_path)
  im_arr.append(im)

output = cv2.VideoWriter("Interpolated"+video_name, 
                         cv2.VideoWriter_fourcc(*'MP4V'),
                         2*fps_og, size)
for image in im_arr:
  output.write(image)
output.release()